<h1>CS4619: Artificial Intelligence 2</h1>
<h2>Model Selection</h2>
<h3>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h3>

# Initialization $\newcommand{\Set}[1]{\{#1\}}$ $\newcommand{\Tuple}[1]{\langle#1\rangle}$ $\newcommand{\v}[1]{\pmb{#1}}$ $\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ $\newcommand{\rv}[1]{[#1]}$

In [13]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

<h1>Model Selection</h1>
<p>
    As discussed previously, the learning algorithm finds the values of parameters. But there are other parameters of
    learning algorithms &mdash; ones that the learning algorithm does not set &mdash; which are called hyperparameters.
    Our best example so far is the value of $k$ in kNN.
</p>
<p> 
    In this lecture, we look at ways of setting the values of the hyperparameters. This is called <b>model
    selection</b>. Despite what it may sound like, this is not about choosing between different
    learning algorithms (e.g. between OLS linear regression and kNN) &mdash; that, after all, can be done by
    error estimation (which we discussed in a previous lecture). Model selection is about optimizing
    the settings for each learning algorithm.
</p>
<p>
    For some learning algorithms there might be special methods for setting some of the hyperparameters.
    If not, in essence, you just try lots of values for your hyperparameters and see which is best &mdash; 
    an approach known as 'grid search'. But how do we estimate the error of each hyperparameter value that we try? 
    This is what we will discuss first.
</p>

<h2>Validation Sets</h2>
<p>
   As in the holdout method, we randomly partition the dataset into disjoint sets. Here, these partitions are
   called the training set and the <b>validation set</b>. 
</p>
<p>
    For each value of the hyperparameter that you wish to try, you train the learning algorithm on the training set. 
    So, assuming one hyperparameter with $\mathit{numvals}$ values that you wish to try, you will learn $\mathit{numvals}$ models. Let's designate
    them by $h_1,\ldots,h_{\mathit{numvals}}$. You test each of them on
    the validation set, e.g. measuring mean squared error, as before. From this, you can see which of the
    $numvals$ models (corresponding to $numvals$ different values of the hyperparameter) appears to be the best &mdash; the
    one with the lowest error on the validation set. This gives you the value of the hyperparameter. That's
    model selection.
</p>
<p>
    The weakness of this (as before) is that the training and validation sets may not be representative: we may get
    'lucky' or 'unlucky'. So we need to use <em>resampling</em> again: either repeated holdout, $k$-fold cross-validation,
    repeated $k$-fold cross-validation, leave-one-out cross-validation, or one of the other methods that we did not study. For
    concreteness, let's focus on one of them: $k$-fold cross-validation.
</p>

<h2>$k$-Fold Cross-Validation for Model Selection</h2>
<p>
    To use $k$-fold cross-validation for model selection, you divide into $k$ folds. You take all but fold 1 as the
    training set and you learn $\mathit{numvals}$ models (one per hyperparameter value), and you test them on fold 1, recording the
    mean squared error of each model. You repeat, taking a different fold to be the test set each time. For each
    hyperparameter value, you take the mean of its $k$ mse's. The hyperparameter value with lowest mean mse is the value
    that you should 'go live' with. (At this point, you can train on <em>all</em> the data, using the best
    hyperparameter value that you have just discovered.) Here it is in pseudocode:
</p>
<ul style="background: lightgray; list-style: none">
    <li>
        partition the dataset $D$ into $k$ disjoint equal-sized subsets, $T_1, T_2,\ldots,T_k$
    <li>
    <li>
        <b>for</b> $u = 1$ to $k$
        <ul>
            <li>
                <b>for</b> $v = 1$ to $\mathit{numvals}$
                <ul>
                    <li>train on $D \setminus T_u$ with the $v$th hyperparameter value</li>
                    <li>make predictions for $T_u$</li>
                    <li>measure error (e.g. MSE)</li>
                </ul>
            </li>
        </ul>
    </li>
    <li>
        get the mean of the errors for each hyperparameter value
    </li>
    <li>
        report the hyperparameter value with lowest mean mse
    </li>
</ul>
<p>
    Let's now combine this validation set idea with grid search &mdash; and let's do so
    in scikit-learn.
</p>

<h2>Grid Search</h2>
<p>
    <b>Grid search</b> is an 'exhaustive' approach &mdash; it tries every value for the hyperparameter (or, at least, all
    values in a certain range). If there is more than one hyperparameter, it tries every combination of their values.
</p>
<ul>
    <li>
        If one hyperprameter has $\mathit{numvals}_1$ values and another has $\mathit{numvals}_2$ values, how many models will grid search train?
    </li>
</ul>

<h2>Grid Search and Holdout in scikit-learn</h2>
<p>
    Let's try all values of kNN's hyperparameter $k$ between 1 and 10 for unweighted kNN on the Cork property dataset using
    holdout (where we simply split the dataset once into training set and validation set):
</p>

In [16]:
# Use pandas to read the CSV file
df = pd.read_csv("dataset-corkA.csv")

# Get the feature-values and the target values into separate numpy arrays of numbers
X = df[['flarea', 'bdrms', 'bthrms']].values
y = df['price'].values

# Create kNN Regressor object
estimator = KNeighborsRegressor()

# Dictionary containing each hyperparameter and the hyperparameter#s possible values 
hyperparameters = {'n_neighbors' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

# We want to split the dataset just once into training and validation
ss = ShuffleSplit(n_splits = 1, test_size = 0.3)

# And here we create the Grid Search object and run its fit method
gs = GridSearchCV(estimator, hyperparameters, scoring = 'neg_mean_squared_error', cv = ss)
gs.fit(X, y)

# Display some of its instance variables afterwards
print(gs.cv_results_)
print(gs.best_estimator_)
print(gs.best_score_)
print(gs.best_params_)

{'std_score_time': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 'rank_test_score': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32), 'mean_train_score': array([  -189.60897436, -28748.9375    , -45408.66381766, -46205.99198718,
       -48393.7374359 , -49434.63817664, -51196.90384615, -52501.96254006,
       -54978.92679645, -56947.94211538]), 'mean_fit_time': array([ 0.00022936,  0.00027466,  0.00019097,  0.00019121,  0.00019288,
        0.0001986 ,  0.00021362,  0.00020933,  0.00026512,  0.00019193]), 'split0_test_score': array([ -13169.20588235,  -38995.32352941,  -59840.95915033,
        -78163.41636029,  -86328.64058824,  -93600.46732026,
       -100748.29231693, -108507.90579044, -116618.70787945,
       -122674.71044118]), 'mean_score_time': array([ 0.00041914,  0.00057435,  0.00038886,  0.00041723,  0.00040364,
        0.00041223,  0.00041723,  0.00046158,  0.00050592,  0.00042892]), 'param_n_neighbors': masked_array(data = [1 2 3 4 5 6 7 8 9 10],
    

<p>
    This suggests a particular value for $k$. But, remember, we are doing only one split of the data. Re-run it and you 
    will see that the best value for the hyperparameter differs from run to run.
</p>

<h2>Grid Search and $k$-Fold Cross-Validation in scikit-learn</h2>
<p>
    If we want to do $k$-fold cross-validation instead (which we hope will give a more robust answer), we simply feed a
    different cross-validation method into the grid search. As we saw before, scikit-learn
    makes it very easy to choose $k$-fold cross-validation: simply supply the number
    of folds (but, remember, you may need to do your own shuffling):
</p>

In [17]:
# Create the Grid Search object and run its fit method
gs = GridSearchCV(estimator, hyperparameters, scoring = 'neg_mean_squared_error', cv = 10)
gs.fit(X, y)

# Display some of its instance variables afterwards
print(gs.cv_results_)
print(gs.best_estimator_)
print(gs.best_score_)
print(gs.best_params_)

{'split2_train_score': array([  -484.09950249, -25589.34452736, -53233.55776672, -51515.16075871,
       -56559.79621891, -58103.73231067, -61479.01319931, -64792.29337687,
       -69132.5631718 , -72318.99691542]), 'split8_test_score': array([-6497.81818182, -4773.09090909, -4879.05050505, -5707.80965909,
       -5204.12727273, -5143.66792929, -4922.05380334, -5193.40767045,
       -4869.08361392, -4660.97772727]), 'split4_train_score': array([  -452.12871287, -13167.84282178, -41085.96644664, -56588.97741337,
       -57259.14950495, -61631.63723872, -62278.2208527 , -64311.56079827,
       -66987.55610561, -70195.60311881]), 'std_score_time': array([  9.78169136e-05,   4.80696793e-05,   6.53671397e-06,
         5.60988328e-06,   9.21924908e-06,   2.36139009e-05,
         3.94680412e-04,   9.46444915e-05,   3.30601212e-05,
         4.02091714e-05]), 'split0_test_score': array([-358085.56521739, -482873.84782609, -370830.08695652,
       -221937.86141304, -144408.27304348,  -98501.7161

<p>
    It selects $k = 9$ (although if we were shuffling the dataset, we might still get different answers each time we run it, but with a lot less
    variability than before).
</p>
<p>
    Assuming we now want to 'go live', we would train 9NN on <em>all</em> the data.
</p>

<h1>Error Estimation, again</h1>
<p>
    Suppose you want to do error estimation, as before, i.e. you want to know how well a model
    is likely to perform on unseen data. But suppose there is a hyperparameter and so you want 
    your estimate of the error on unseen data to use a good value for the hyperparameter. In this case, we need to combine
    ideas from the earlier part of this lecture with ideas from the error estimation lecture.
</p>
<p>
    The simplest case is to partition your data into three sets (assuming you have enough data!):
</p>
<ul>
    <li>
        training set,
    </li>
    <li>
        validation set, and
    </li>
    <li>
        test set.
    </li>
</ul>
<p>
    You use the training set and the validation set to choose the best value for the hyperparameter, i.e. train $\mathit{numvals}$
    models (one for each value of the hyperparameter), test them on the validation set, using grid search.
    Choose the value that gave
    the lowest mse. Then, test the model you have just selected on the test set: the mse that this produces is your
    estimate of the error on unseen examples.
</p>

<h2>Training Set, Validation Set and Test Set in scikit-learn</h2>
<p>
    In this scikit-learn example, we will split the data 60-20-20:
</p>

In [18]:
# Split the data 80/20: the 80 is the combined training and validation sets; the 20 is the final test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2)

# Split the combined set just once into training and validation: 75/25 gives 60/20 of the original data
ss = ShuffleSplit(n_splits = 1, test_size = 0.25, random_state = np.random)

# And here we create the Grid Search object and run its fit method
gs = GridSearchCV(estimator, hyperparameters, scoring = 'neg_mean_squared_error', cv = ss)
gs.fit(X_train_val, y_train_val)

# Take the best estimator found by the grid search
# Test it on the test set 
y_predicted = gs.predict(X_test)
mse_test = mean_squared_error(y_test, y_predicted)

# Display
mse_test

181434.79288888889

<p>
    This estimate has the weakness (again) that it is based on just one way of partitioning the data: different partitions
    will have very different error estimates &mdash; as you can confirm by running the code seveal times.
    We need to use resampling.
</p>

<h2>Nested $k$-Fold Cross-Validation</h2>
<p>
    What we will do is called <b>nested $k$-fold cross validation</b>. As usual, it partitions the dataset into $k$ folds.
    One fold is kept as the test set. The remaining data is then partitioned again into $k$ folds. One of <em>these</em>
    folds is kept as the validation set. The remaining is used for training. We learn $\mathit{numvals}$ models from the training data
    (one for each value of the hyperparameter). We test each of them on the validation set. We repeat this, taking each
    <em>inner fold</em> in turn as the validation set. When we have done, we can select the best value of the hyperparameter.
    We test this model on the <em>outer fold</em> that we kept as test set. Then the whole thing repeats: a different outer
    fold becomes the test set; $k$-fold cross-validation is run on the remaining data to find another best value for
    the hyperparameter; and this model is tested on the outer fold that was kept as test set. And so on.
</p>
<p>
    In pseudocode:
</p>
<ul style="background: lightgray; list-style: none">
    <li>
        partition the dataset $D$ into $k$ disjoint equal-sized subsets, $T_1, T_2,\ldots,T_k$
    <li>
    <li>
        <b>for</b> $u_1 = 1$ to $k$
        <ul>
            <li>
                let $D'$ be $D \setminus T_{u_1}$
            </li>
            <li>
                partition $D'$ into $k$ disjoint equal-sized subsets, $S_1, S_2,\ldots,S_k$
            </li>
            <li>
                <b>for</b> $u_2 = 1$ to $k$
                <ul>
                    <li>
                        <b>for</b> $v = 1$ to $\mathit{numvals}$
                        <ul>
                            <li>train on $D' \setminus S_{u_2}$ with the $v$th hyperparameter value</li>
                            <li>make predictions for $S_{u_2}$</li>
                            <li>measure validation error (e.g. MSE)</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>
                get the mean of the errors for each hyperparameter value
            </li>
            <li>
                select the model (hyperparameter value) with lowest mean mse
            </li>
            <li>
                use the selected model to make predictions for $T_{u_1}$
            </li>
            <li>
                measure test error (e.g. MSE)
            </li>
        </ul>
    </li>
    <li>
        report the means of the test errors
    </li>
</ul>

<h2>Nested $k$-Fold Cross-Validation in scikit-learn</h2>
<p>
    This is the short version again (but again you should probably shuffle the dataset first):
</p>

In [19]:
gs = GridSearchCV(estimator, hyperparameters, scoring = 'neg_mean_squared_error', cv = 10)
mses_test = cross_val_score(gs, X, y, scoring = 'neg_mean_squared_error', cv = 10)
mean_mse_test = np.mean(mses_test)
mean_mse_test

-101001.56507719708

<p>
    Of course, our little dataset really isn't big enough for this kind of treatment.
</p>

<h2>Confusion</h2>
<p>
    You might be thinking to yourself: "But the best value for the hyperparameter can be different for each outer fold." You are
    correct. And so then you may be thinking: "So which value for the hyperparameter do I choose?" To which the reply is: 
    "None of them. This is not model selection. This is just error estimation." If you want to do model selection, see
    earlier.
</p>